In [1]:
!pip install git+https://github.com/huggingface/accelerate
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install gdown
!pip install wandb

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-hdh6m10h
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-hdh6m10h
  Resolved https://github.com/huggingface/accelerate to commit ab379793d44be16d8fcac5c098a3ab9b6f5a7ec3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.20.0.dev0-py3-none-any.whl size=222939 sha256=ad54284ce01fc38314ae96595efda530bea2aa027b084a794ebb35d7d0c04bdf
  Stored in directory: /tmp/pip-ephem-wheel-cache-d7nabg9c/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0


In [2]:
# import os
# print(os.environ.get("LOCAL_RANK",0))

In [3]:
import json
import os
from typing import Union,List
import sys

import torch
from transformers import XGLMTokenizer, XGLMForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from datasets import load_dataset
import multiprocessing

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7"

In [5]:
cpu_cores = multiprocessing.cpu_count()

In [7]:
def load_model(base_model:str="facebook/xglm-564M"):
    tokenizer = XGLMTokenizer.from_pretrained(base_model)
    model = XGLMForCausalLM.from_pretrained(base_model)
    
    new_tokens = ['<human>:', '<bot>:']
    tokenizer.add_tokens(list(new_tokens))
    model.resize_token_embeddings(len(tokenizer))
    return model,tokenizer
model,tokenizer = load_model()

In [8]:
# model.hf_device_mapmodel

In [9]:
import gdown

url = 'https://drive.google.com/uc?export=download&id=1jbbUtwgwoSQgGnXxzTh-nMReVzEU7ZTU&confirm=t&uuid=d79e2e78-51de-466f-9ceb-3944606141a2&at=AKKF8vwcgi95TGSnSQUNCKx4NTqS:1682865249145'
output = 'output.jsonl'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1jbbUtwgwoSQgGnXxzTh-nMReVzEU7ZTU&confirm=t&uuid=d79e2e78-51de-466f-9ceb-3944606141a2&at=AKKF8vwcgi95TGSnSQUNCKx4NTqS:1682865249145
To: /kaggle/working/output.jsonl
100%|██████████| 167M/167M [00:00<00:00, 176MB/s] 


'output.jsonl'

In [10]:
def format_prompt(prompt):
    text = {'prompt':f"{prompt['Background:']} <human>: {prompt['<human>:']} <bot>: {prompt['<bot>:']}"
           }
    text['token_prompt'] = len(tokenizer.tokenize(text['prompt']))
    return text
def find(prompt):
    return prompt['token_prompt']<254
def preprocess(prompt):
    inputs = tokenizer(
        prompt['prompt'],
        truncation=True,
        max_length=256,
        padding=False,
        return_tensors=None,
    )
    inputs['input_ids'].append(tokenizer.eos_token_id)
    inputs['attention_mask'].append(1)
    inputs['labels'] = inputs['input_ids'].copy()
    return inputs

In [11]:
from datasets import load_dataset
datasets = load_dataset('json',data_files = 'output.jsonl')
datasets

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-9ebe4cd6c3924f90/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Background:', '<human>:', '<bot>:'],
        num_rows: 116288
    })
})

In [12]:
from datasets import Dataset
datasets = Dataset.from_dict(datasets['train'][:1_000]) # sample data for test
# datasets = datasets['train']
datasets = datasets.map(format_prompt,remove_columns=['Background:', '<human>:', '<bot>:'],num_proc=cpu_cores)
datasets = datasets.filter(find,num_proc=cpu_cores) # two for <\s> token
datasets = datasets.map(preprocess,remove_columns=['prompt','token_prompt'],num_proc=cpu_cores) 
print(datasets)

#0:   0%|          | 0/500 [00:00<?, ?ex/s]

#1:   0%|          | 0/500 [00:00<?, ?ex/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/252 [00:00<?, ?ex/s]

#1:   0%|          | 0/251 [00:00<?, ?ex/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 503
})


In [13]:
print(tokenizer.decode(datasets[0]['input_ids']))
# print(tokenizer.decode(datasets[0]['labels']))

</s> Tubulin in molecular biology can refer either to the tubulin protein superfamily of globular proteins or one of the member proteins of that superfamily α and βtubulins polymerize into microtubules a major component of the eukaryotic cytoskeleton Microtubules function in many essential cellular processes including mitosis Tubulinbinding drugs kill cancerous cells by inhibiting microtubule dynamics which are required for DNA segregation and therefore cell division there are six members of the tubulin superfamily although not all are present in all species Both α and β tubulins have a mass of around 50 kDa and are thus in a similar range compared to actin with a mass of 42 kDa In contrast tubulin polymers microtubules tend to be much bigger than actin filaments due to their cylindrical nature Tubulin was long thought to be specific to eukaryotes More recently however several prokaryotic proteins have been shown to be related to tubulin <human>: What are some examples of drugs that bi

In [14]:
batch_size = 128
micro_batch_size = 4
gradient_accumulation_steps = batch_size // micro_batch_size
num_epochs = 3
learning_rate = 3e-7
output_dir = 'checkpoint-xglm'

In [15]:
os.environ["WANDB_PROJECT"] = 'wandb_project'
os.environ["WANDB_LOG_MODEL"] = 'true'

In [17]:
train_args = TrainingArguments( # สร้าง class train-args
            per_device_train_batch_size=micro_batch_size, # btch_size 
            gradient_accumulation_steps=gradient_accumulation_steps, # https://huggingface.co/docs/accelerate/usage_guides/gradient_accumulation เหมือนจะ ค่อยๆคำนวนค่า gradient ตามค่าที่ใส่เข้าไปรอบ แล้วค่อยปรับ weight ทีเดียว ไม่รู้_
#             gradient_checkpointing=True,
            warmup_steps=100,# ไม่รู้_
            num_train_epochs=num_epochs, # จำนวน epoch
            learning_rate=learning_rate,# ค่า learning-rate
            fp16= True, # ไม่รู้ว่าคืออะไร ตอนแรก default คือ True เลยเปลี่ยนเป็น False แทน แล้วรันได้เฉย _   ///////// เพราะเราเซ้ตข้างบนไว้ว่าเป็น torch.float16 
            logging_steps=1, # ไม่รู้_ //////////// แสดงผลตอนเทรนทุกๆ 10 step gradient descent
            optim="adamw_torch",# ชื่อ optimizer มั้ง_ /////// yes!!
            save_strategy="steps", # ไม่รู้_ //////////////////////////////// save model based on epoch? steps?
            save_steps=1_000, # ไม่รู้_ ///////////////////// Save model every 200 optimizer.step()
            output_dir=output_dir, # ไม่รู้_ ////////////////////////// Where to save model
            save_total_limit=3, # ไม่รู้_ /////////////////////////// Limit model save amount (Not to have 300 model file when you train 300 epoch)
            report_to="wandb", # ใช้ wandb
            run_name='finetune-xglm', # ชื่อ task
        )

trainer = Trainer(
      model=model,# model ที่จะเอาไปเทรน
      train_dataset=datasets, # data ใน train-set
      # eval_dataset=val_data,
      args=train_args,
      data_collator=DataCollatorForSeq2Seq(
          tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True 
      ),
  )

# model.config.use_cache = False

# train-ai ปกติ
trainer.train()
trainer.save_model()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_31/4024636513.py:32 in <module>                                                   │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_31/4024636513.py'                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:1662 in train                    │
│                                                                                                  │
│   1659 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1660 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1661 │   │   )                                                                                 │
│ ❱ 1662 │   │   return inner_training_loop(                                                       │
│   1663 │   │   │   args=args,                                                                    │
│   1664 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1665 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:1929 in _inner_training_loop     │
│                                                                                                  │
│   1926 │   │   │   │   │   with model.no_sync():                                                 │
│   1927 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1928 │   │   │   │   else:                                                                     │
│ ❱ 1929 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1930 │   │   │   │                                                                             │
│   1931 │   │   │   │   if (                                                                      │
│   1932 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:2699 in training_step            │
│                                                                                                  │
│   2696 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2697 │   │                                                                                     │
│   2698 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2699 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2700 │   │                                                                                     │
│   2701 │   │   if self.args.n_gpu > 1:                                                           │
│   2702 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:2731 in compute_loss             │
│                                                                                                  │
│   2728 │   │   │   labels = inputs.pop("labels")                                                 │
│   2729 │   │   else:                                                                             │
│   2730 │   │   │   labels = None                           